In [1]:
import pandas as pd
import pandas_datareader as data
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime, date

In [2]:
start_date = '1928-01-01'
end_date = date.today()
MAX_YEARS = 40
TAX = 0.2
INIT_AMOUNT = 0
MAX_AMOUNT = 20000000
step = 100000
monthly_min = 3000
monthly_max = 15000
monthly_step = 1000
min_amount_tolerable = 100000
INFLATION_ON = True
dump_data = True
data_source = 'yahoo'
default_model = 'S&P500'
MODELS = {'S&P500':'^GSPC','NASDAQ':'^IXIC','DOW':'^DJI','Tres10y':'^TNX','Coke':'KO','GE':'GE','IBM':'IBM'}

result_df = pd.DataFrame(columns = ['start_date','principle','monthly','months_survive','survive']).astype({'survive': bool})

In [16]:
#validate code
import yfinance as yf
#yf.pdr_override()
ydf = yf.download('^GSPC', start_date, end_date)

ydf.columns = ydf.columns.get_level_values(0)
ydf[['Close']]
#data.get_data_yahoo(['^IXIC'], start=start_date, end=end_date)

[*********************100%***********************]  1 of 1 completed


Price,Close
Date,
1928-01-03,17.760000
1928-01-04,17.719999
1928-01-05,17.549999
1928-01-06,17.660000
1928-01-09,17.500000
...,...
2025-02-11,6068.500000
2025-02-12,6051.970215
2025-02-13,6115.069824


In [24]:
def prep_data(source=MODELS[default_model], inflation=INFLATION_ON ):
    print('prep started')
    # pull raw data
    #stock = data.DataReader(source, data_source, start_date, end_date)[['Close']]
    ydf = yf.download(source, start_date, end_date)
    ydf.columns = ydf.columns.get_level_values(0)
    stock = ydf[['Close']].copy()
    ydf = pd.DataFrame()
    #pull the earliest stock date available
    new_start = stock.index[0]
    cpi = data.DataReader("CPIAUCNS", "fred", new_start, end_date)
    
    # user merger and then fill in monthly from the previous close if possible else the next close
    df = cpi.merge(stock, how='outer', left_index=True, right_index=True)
    df[['Close']] = df[['Close']].ffill().bfill()

    # remove non first month 
    df.dropna(inplace=True)

    # set placeholders
    df['inflation'] = 0.0
    df['change'] = 0.0
    df.reset_index(inplace=True)

    # calculate baseline df
    for i in range(len(df)):
        # skip the first iteration
        if i == 0: continue
        # update inflation
        if inflation:
            p_cpi = df.at[i-1,'CPIAUCNS']
            c_cpi = df.at[i,'CPIAUCNS']
            df.at[i,'inflation']= (c_cpi - p_cpi)/p_cpi
        # update percent change
        p_close = df.at[i-1,'Close']
        c_close = df.at[i,'Close']
        df.at[i,'change'] = (c_close - p_close)/p_close
    print('prep done')
    return df

In [25]:
prep_data()

[*********************100%***********************]  1 of 1 completed

prep started


prep done


,index,CPIAUCNS,Close,inflation,change
0,1928-02-01,17.100,17.530001,0.000000,0.000000
1,1928-03-01,17.100,17.299999,0.000000,-0.013120
2,1928-04-01,17.100,19.280001,0.000000,0.114451
3,1928-05-01,17.200,19.780001,0.005848,0.025934
4,1928-06-01,17.100,20.070000,-0.005814,0.014661
...,...,...,...,...,...
1159,2024-09-01,315.301,5648.399902,0.001604,0.037035
1160,2024-10-01,315.664,5708.750000,0.001151,0.010684
1161,2024-11-01,315.493,5728.799805,-0.000542,0.003512
1162,2024-12-01,315.605,6032.379883,0.000355,0.052992


In [26]:
def model_year(prin, start_date_index, withdrawal, min_amount_tol, df, tax=TAX, years=MAX_YEARS):
    global result_df
    # calculates the model year and inserts into results df
    # returns boolean for making it to the end of the year
    failed = False
    s_date = df.at[start_date_index,'index'].strftime('%Y_%m_%d')
    mnth = f'monthly_{s_date}_{prin}_{withdrawal}'
    prcp = f'principle_{s_date}_{prin}_{withdrawal}'
    df[mnth] = np.nan
    df[prcp] = np.nan
    k=0
    end = start_date_index + years*12 +1
    start = df.at[start_date_index,'index']
    for i in range(start_date_index,end):
        if k==0:
            df.at[i,mnth]= withdrawal
            df.at[i,prcp]= prin
            k+=1
            continue
        if i >= len(df):
            break
        p = i-1
        # previous monthly * (1+inflation)
        p_m = df.at[p,mnth]
        c_i = df.at[i,'inflation']
        c_m = p_m*(1+c_i)
        df.at[i,mnth]= c_m
        # (previous principle*(1+change)) - current monthly
        p_p = df.at[p,prcp]
        p_c = df.at[p,'Close']
        c_c = df.at[i,'Close']
        change =  (c_c-p_c)/p_c
        c_p = p_p*(1+change)-(c_m/(1-tax))
        df.at[i,prcp]= c_p
        failed = (c_p <= min_amount_tol)
        if failed or i == end-1:
            res = pd.DataFrame.from_dict(
                {'start_date': [start], 
                 'principle': prin, 
                 'monthly': withdrawal,
                 'months_survive': [i-start_date_index], 
                 'survive': [not failed]}).astype({'survive': bool})
            result_df = pd.concat([result_df,res], ignore_index=True)
            break
    if dump_data:
        df.drop(columns=[mnth,prcp], inplace=True)
    return failed
    

In [ ]:
# Testing
# model_year(init_amount, 0, 8000, df)
# result_df.head()

In [ ]:
def seek_year(sdi, p_min, p_max, step, withdrawal, min_amount_tol, baseline_df, tax=TAX, years=MAX_YEARS):
    # sdi = start date index 
    # p_min = principle min
    # p_max = principle max
    # step = increase step from in to max
    
    # presumes failed state
    failed = True
    for prin in range(p_min,p_max,step):
        failed = model_year(prin, sdi, withdrawal, min_amount_tol, baseline_df, tax, years)
        # exists when succeeds
        if not failed:
            break

In [ ]:
# Testing
# seek_year(0, init_amount, max_amount, step, monthly,df)
# result_df.head()

In [ ]:
def run_model(model = MODELS['S&P500'], years=MAX_YEARS,
              min_amount_tol=min_amount_tolerable, tax=TAX,
              monthly_min= monthly_min, monthly_max=monthly_max, inflation= INFLATION_ON):
    model_df = prep_data(model)
    lsi = int((len(model_df)- years*12)/12)
    for s in range(lsi + 1):
        s_date_i = s*12
        year = model_df.at[s_date_i,'index'].strftime('%Y_%m_%d')
        print(f'analyzing year: {year} {s+1} of {lsi + 1}')
        for monthly in range(monthly_min,monthly_max+monthly_step,monthly_step):
            seek_year(s_date_i, INIT_AMOUNT, MAX_AMOUNT, step, monthly, min_amount_tol, model_df,tax, years)

In [ ]:
def only_survived():
    survive_df = result_df[result_df.survive]
    survive_df= survive_df.astype({'start_date': object, 
                                   'principle': int, 
                                   'monthly': int, 
                                   'months_survive': int, 
                                   'survive': bool})
    return survive_df

In [ ]:
%%time
run_model()
sdf = only_survived()

In [ ]:
sdf

In [ ]:
inf_text = 'Yes' if INFLATION_ON else 'No'
title = f'principle needed (inflation {inf_text}, min_bal {min_amount_tolerable}, years {MAX_YEARS}, model {default_model})'
x_l = 'monthly withdrawal'
y_l = 'principle'

unique_monthly = sdf.monthly.unique()
data =[list(sdf[sdf['monthly'] == i].principle) for i in unique_monthly]
fig, ax = plt.subplots()

fig.set_size_inches(10, 6)
ax.set_title(title)
ax.boxplot(data, showmeans=True)
ax.set_xticklabels(['${:,}'.format(x) for x in unique_monthly])
ax.yaxis.set_major_formatter('${x:,.0f}')
ax.set_ylabel(y_l)
ax.set_xlabel(x_l)

r

plt.show()


In [ ]:
list(sdf.monthly.unique())

In [ ]:
sdf2 = sdf[['principle','monthly']].groupby('monthly').describe()

